In [1]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv
/kaggle/input/new-york-city-taxi-fare-prediction/GCP-Coupons-Instructions.rtf
/kaggle/input/new-york-city-taxi-fare-prediction/train.csv
/kaggle/input/new-york-city-taxi-fare-prediction/test.csv


### Setp 1
- Install req libs and Load the datas

In [2]:
import numpy as np
import pandas as pd

In [3]:
data_dir = '/kaggle/input/new-york-city-taxi-fare-prediction/'

In [4]:
!ls -lh {data_dir}

total 5.4G
-rw-r--r-- 1 nobody nogroup  486 Apr  4 03:51 GCP-Coupons-Instructions.rtf
-rw-r--r-- 1 nobody nogroup 336K Apr  4 03:51 sample_submission.csv
-rw-r--r-- 1 nobody nogroup 960K Apr  4 03:51 test.csv
-rw-r--r-- 1 nobody nogroup 5.4G Apr  4 03:53 train.csv


All the data is in the **train.csv** file

In [5]:
# this will take too much tine
# !wc -l {data_dir}/train.csv

In [6]:
!wc -l {data_dir}/test.csv

9914 /kaggle/input/new-york-city-taxi-fare-prediction//test.csv


Getting the first line of the dataset using the _shell_ command

In [7]:
    !head {data_dir}/train.csv

In [8]:
!head {data_dir}/test.csv

fare amount is missingw which is the pratical Target column for our analysis

#### Loading
Loading the entire dataset into Pandas is going to be slow, so we can use the following optimizations:

- Ignore the key column
- Parse pickup datetime while loading data
- Specify data types for other columns
    - float32 for geo coordinates
    - float32 for fare amount
    - uint8 for passenger count
- Work with a 1% sample of the data (~500k rows)

In [9]:
# Taking all the columns except the keys
selected_cols = 'fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count'.split(',')
selected_cols

['fare_amount',
 'pickup_datetime',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'passenger_count']

In [10]:
d_types = {
    'fare_amount' : 'float32',
    'pickup_longitude': 'float32',
    'pickup_latitude': 'float32',
    'dropoff_longitude': 'float32',
    'dropoff_latitude' : 'float32',
    'passenger_count' : 'uint8',
}
d_types

{'fare_amount': 'float32',
 'pickup_longitude': 'float32',
 'pickup_latitude': 'float32',
 'dropoff_longitude': 'float32',
 'dropoff_latitude': 'float32',
 'passenger_count': 'uint8'}

In [11]:
import random as rnd
sample_fraction = 0.01
rnd.seed(42)

def skip_row(row_idx) :
#     If this functon retrun false then the number is going to be in data
    if row_idx == 0 :
        return False
    return rnd.random() > sample_fraction

In [12]:
data = pd.read_csv(data_dir + '/train.csv', 
                   usecols = selected_cols, 
                   dtype = d_types, 
                   parse_dates= ['pickup_datetime'], 
                   skiprows = skip_row) # A function is passed as a prameter

In [13]:
data.sample(10)

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
153511,10.500000,2013-09-25 19:07:00+00:00,-73.963371,40.754353,-73.978195,40.766560,1
361285,17.700001,2010-04-24 11:41:40+00:00,-73.979706,40.786533,-73.988014,40.737659,1
287793,4.900000,2012-04-08 06:01:08+00:00,-73.986526,40.755856,-73.998604,40.764027,1
85309,7.000000,2013-05-24 22:17:00+00:00,-73.990189,40.756420,-73.992081,40.746723,2
67448,49.150002,2009-03-30 15:41:00+00:00,-73.786720,40.641708,-73.974976,40.755531,1
370389,16.500000,2010-04-19 17:03:55+00:00,-74.007668,40.703640,-73.989838,40.750771,1
38648,4.500000,2011-10-14 20:14:09+00:00,-73.981628,40.732533,-73.984169,40.740334,1
210596,4.900000,2012-05-08 20:58:58+00:00,-73.953278,40.776825,-73.952789,40.768173,1
535651,4.500000,2015-06-05 07:46:40+00:00,-73.959709,40.762672,-73.953888,40.770760,1
416843,10.100000,2009-12-27 00:22:24+00:00,-73.991417,40.750790,-73.990791,40.717628,1


Here I am trying with 1% if the data, We can also try this using _3%, 10%, 50% and even 100%_ of the data

In [14]:
test_data = pd.read_csv(data_dir + '/test.csv', dtype= d_types, parse_dates = ['pickup_datetime'])
test_data.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24+00:00,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24+00:00,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44+00:00,-73.982521,40.751259,-73.979652,40.746140,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12+00:00,-73.981163,40.767807,-73.990448,40.751637,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12+00:00,-73.966049,40.789776,-73.988564,40.744427,1


### Exploring the data

- basic info about training and test set
- EDA and viz
- Ask and Answer the questions

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552450 entries, 0 to 552449
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   fare_amount        552450 non-null  float32            
 1   pickup_datetime    552450 non-null  datetime64[ns, UTC]
 2   pickup_longitude   552450 non-null  float32            
 3   pickup_latitude    552450 non-null  float32            
 4   dropoff_longitude  552450 non-null  float32            
 5   dropoff_latitude   552450 non-null  float32            
 6   passenger_count    552450 non-null  uint8              
dtypes: datetime64[ns, UTC](1), float32(5), uint8(1)
memory usage: 15.3 MB


In [16]:
data.shape

(552450, 7)

In [17]:
data.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,552450.000000,552450.000000,552450.000000,552450.000000,552450.000000,552450.000000
mean,11.354059,-72.497063,39.910500,-72.504326,39.934265,1.684983
std,9.811924,11.618246,8.061114,12.074346,9.255057,1.337664
min,-52.000000,-1183.362793,-3084.490234,-3356.729736,-2073.150635,0.000000
25%,6.000000,-73.992020,40.734875,-73.991425,40.733990,1.000000
50%,8.500000,-73.981819,40.752621,-73.980179,40.753101,1.000000
75%,12.500000,-73.967155,40.767036,-73.963737,40.768059,2.000000
max,499.000000,2420.209473,404.983337,2467.752686,3351.403076,208.000000


In [18]:
data.pickup_datetime.min(), data.pickup_datetime.max()

(Timestamp('2009-01-01 00:11:46+0000', tz='UTC'),
 Timestamp('2015-06-30 23:59:54+0000', tz='UTC'))

In [19]:
test_data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   key                9914 non-null   object             
 1   pickup_datetime    9914 non-null   datetime64[ns, UTC]
 2   pickup_longitude   9914 non-null   float32            
 3   pickup_latitude    9914 non-null   float32            
 4   dropoff_longitude  9914 non-null   float32            
 5   dropoff_latitude   9914 non-null   float32            
 6   passenger_count    9914 non-null   uint8              
dtypes: datetime64[ns, UTC](1), float32(4), object(1), uint8(1)
memory usage: 319.6+ KB


The data is not cleaned as -1000 Latitude does not exist and also _208_ pasengers are can't ride a Taxi

#### Observations
- 550K+ data
- No missing data (in sample)
- Fare ranges from -52 to 499
- Pasengers from 0 to 208
- Seems to be error in the latitue and logitude
- Dates Ranges from 1st Jan 2009 to 30th June 2015

Need to deal with the outliares and preproces the data.

In [20]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   key                9914 non-null   object             
 1   pickup_datetime    9914 non-null   datetime64[ns, UTC]
 2   pickup_longitude   9914 non-null   float32            
 3   pickup_latitude    9914 non-null   float32            
 4   dropoff_longitude  9914 non-null   float32            
 5   dropoff_latitude   9914 non-null   float32            
 6   passenger_count    9914 non-null   uint8              
dtypes: datetime64[ns, UTC](1), float32(4), object(1), uint8(1)
memory usage: 319.6+ KB


In [21]:
test_data.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000
mean,-73.974716,40.751041,-73.973656,40.751740,1.671273
std,0.042774,0.033541,0.039072,0.035435,1.278747
min,-74.252190,40.573143,-74.263245,40.568974,1.000000
25%,-73.992500,40.736125,-73.991249,40.735253,1.000000
50%,-73.982327,40.753052,-73.980015,40.754065,1.000000
75%,-73.968012,40.767113,-73.964062,40.768757,2.000000
max,-72.986534,41.709557,-72.990967,41.696682,6.000000


In [22]:
test_data.pickup_datetime.min(), test_data.pickup_datetime.max()

(Timestamp('2009-01-01 11:04:24+0000', tz='UTC'),
 Timestamp('2015-06-30 20:03:50+0000', tz='UTC'))

Training data is Clean we can refer to the taining data for the purpost of clenaing the data.

Like we can remove the columns whose values lies outside the Training data.

## Exploratory Data Analysis

In [23]:
import matplotlib.pyplot as plt
import seaborn as sns

Skipping for Now -> Following the content of the Video This will be done a bit later when working properly

### Preparing the Dataset for training
- Split it into the training and validation set
- Fill/Remove Missing values
- Extrarct Input and Output
    - Training
    - Validation
    - Test

#### Splitting the dataset

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
train, val = train_test_split(data, test_size = 0.2, random_state = len(data))
len(train), len(val)

(441960, 110490)

In [26]:
train.shape

(441960, 7)

In [27]:
# Dropping the missing values
trian = train.dropna()
val = val.dropna()

train.shape, val.shape

((441960, 7), (110490, 7))

Sepraing the inputs and output cols

In [28]:
train.columns

Index(['fare_amount', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'passenger_count'],
      dtype='object')

We can't Train a a model on date time column we need to convert the date time to multiple columns by extraicting the values

In [29]:
input_col = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count']
tar_col = 'fare_amount'

In [30]:
x_train = train[input_col].copy()
y_train = train[tar_col].copy()

x_val = val[input_col].copy()
y_val = val[tar_col].copy()

x_train.shape, y_val.shape

((441960, 5), (110490,))

## Hardcoded and baseline Models

Making some of the very simple models which produce very dumb results and compare the models to the This to know the status
and perfoming efficieny of the modle

In [31]:
# Using Classes to make it similar to the SK-learn models

class MeanRegressor:
    def fit(self, inputs, targets):
        self.mean = targets.mean()
        return self
    
    def predict(self, inputs):
        return np.full(inputs.shape[0], self.mean)
    

In [32]:
model = MeanRegressor().fit(x_train, y_train)

In [33]:
model.mean

11.351758

In [34]:
train_pre = model.predict(x_train)

### Evaluating 
using RMSE for the evaluating the error

In [35]:
from sklearn.metrics import mean_squared_error

def rmse(inp, tar):
    return mean_squared_error(inp, tar, squared=False)

In [36]:
train_rmse = rmse(train_pre, y_train)
train_rmse

9.809799

A very high error in predicting the rate

In [37]:
val_rmse = rmse(model.predict(x_val), y_val)
val_rmse

9.820382

### Train and Evaluate Base line Models

In [38]:
from sklearn.linear_model import LinearRegression

In [39]:
model = LinearRegression().fit(x_train, y_train)

In [40]:
train_pre = model.predict(x_train)
train_pre

array([11.27413 , 11.274368, 11.654195, ..., 11.274407, 11.654392,
       11.654364], dtype=float32)

In [41]:
train_rmse = rmse(train_pre, y_train)
train_rmse

9.808442

In [42]:
model.coef_

array([ 0.00041984, -0.00824938,  0.00203213, -0.00225181,  0.09508976],
      dtype=float32)

In [43]:
val_pre = model.predict(x_val)
val_rmse = rmse(val_pre, y_val)
val_rmse

9.819258

Not at all a good model -> almost equal to the hardcoded model

The main reason for this is Taining data have geo-coordinates, which is not in a format useful for the model and we are not using the one of the most imp column : pickup date and time